In [1]:
import pandas as pd
import os
import numpy as np
#import scikit-learn as sk
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder_path = "/content/drive/MyDrive/UMich/"
train_df = pd.read_csv(os.path.join(folder_path, "train_df_name_enc.csv"))
test_df = pd.read_csv(os.path.join(folder_path, "test_df_name_enc.csv"))

In [4]:
train_df.head()

,zip5,beds,price,BCBS,Medicare,Medicaid,UHC,Aetna,Cigna,Cash Price,...,name_PAUL OLIVER MEMORIAL HOSPITAL,name_PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,name_PROMEDICA COLDWATER REGIONAL HOSPITAL,name_SINAI-GRACE HOSPITAL,name_UP HEALTH SYSTEM - MARQUETTE,name_UP HEALTH SYSTEM PORTAGE,name_VIBRA HOSPITAL OF SOUTHEASTERN MI-TAYLOR CAMPUS,inoutpatient_INPATIENT,inoutpatient_OUTPATIENT,inoutpatient_UNSPECIFIED
0,49007,380,195.70,0,0,1,0,1,0,0,...,False,False,False,False,False,False,False,False,True,False
1,49007,380,450.48,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
2,49007,380,375.16,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False
3,49007,380,1769.16,0,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,True,False
4,49007,380,1769.16,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,False


In [8]:
print(list(train_df.columns))

['zip5', 'beds', 'price', 'BCBS', 'Medicare', 'Medicaid', 'UHC', 'Aetna', 'Cigna', 'Cash Price', 'Gross Charge', 'HospitalInsurance', 'birch_cluster', 'payer_AETNA', 'payer_AETNA - Commercial', 'payer_AETNA - PPO', 'payer_AETNA / COVENTRY', 'payer_AETNA 120101 120101 AETNA MEDICARE ADVANTAGE 601', 'payer_AETNA 120101 120101 AETNA MEDICARE ADVANTAGE 615', 'payer_AETNA 120116 120116 AETNA BETTER HLTH MI 601', 'payer_AETNA 120116 120116 AETNA BETTER HLTH MI 615', 'payer_AETNA 1747', 'payer_AETNA 210109 210109 AETNA BETTER HLTH MI CAID 630', 'payer_AETNA 210109 210109 AETNA BETTER HLTH MI CAID 676', 'payer_AETNA 210120 210120 AETNA BET HLT MI CAID HMP 630', 'payer_AETNA 210120 210120 AETNA BET HLT MI CAID HMP 676', 'payer_AETNA 4008 AETNA ADVANTAGE 553', 'payer_AETNA 4008 AETNA ADVANTAGE 658', 'payer_AETNA 4070 AETNA BTR HLTH MI HLTH LNK 554', 'payer_AETNA 4070 AETNA BTR HLTH MI HLTH LNK 660', 'payer_AETNA 5017 AETNA BTR HLTH MEMIHEAL 715', 'payer_AETNA 5085 AETNA BTR HLTH MEMIMED1 715', '

In [ ]:
def clean_column_names(df):
    df.columns = df.columns.str.replace(r'[\[\]<]', '', regex=True)
    return df

# Clean column names for train_df and test_df
train_df = clean_column_names(train_df)
test_df = clean_column_names(test_df)

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the data
X_train = train_df.drop(columns=['price'])  # Replace 'price' with your target column name
y_train = train_df['price']
X_test = test_df.drop(columns=['price'])
y_test = test_df['price']

# Initialize the XGBoost Regressor
xgb_regressor = XGBRegressor(
    random_state=42,
    n_estimators=100,  # Number of boosting rounds
    learning_rate=0.1,  # Step size shrinkage
    max_depth=6,  # Maximum depth of a tree
    objective='reg:squarederror'  # Regression objective
)
# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions
y_pred = xgb_regressor.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE: {rmse}")

# Optional: Feature Importance
importances = xgb_regressor.feature_importances_
feature_names = X_train.columns
for name, importance in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {importance:.4f}")



In [4]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.metrics import make_scorer

def clean_column_names(df):
    df.columns = df.columns.str.replace(r'[\[\]<]', '', regex=True)
    return df

# Clean column names for train_df and test_df
train_df = clean_column_names(train_df)
test_df = clean_column_names(test_df)

from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the data
X_train = train_df.drop(columns=['price'])  # Replace 'price' with your target column name
y_train = train_df['price']
X_test = test_df.drop(columns=['price'])
y_test = test_df['price']

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1],
}

# Custom RMSE scoring function
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# Initialize the XGBoost Regressor
xgb_regressor = XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Grid Search with Cross Validation
grid_search = GridSearchCV(
    estimator=xgb_regressor,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Fit the grid search
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 1],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 6, 9], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False),
             verbose=2)

In [5]:
# Extract the results
results = pd.DataFrame(grid_search.cv_results_)

# Add RMSE to results
results['mean_test_rmse'] = -results['mean_test_score']  # Negate because scoring was negative RMSE

# Select and save key columns
key_columns = ['param_n_estimators', 'param_learning_rate', 'param_max_depth',
               'param_subsample', 'param_colsample_bytree', 'param_gamma', 'mean_test_rmse']
results = results[key_columns]

# Sort results by RMSE
results = results.sort_values(by='mean_test_rmse').reset_index(drop=True)

# Save results to a DataFrame
results_file = "grid_search_results.csv"
results.to_csv(results_file, index=False)
print(f"Grid search results saved to {results_file}")

# Print the best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", -grid_search.best_score_)

Grid search results saved to grid_search_results.csv
Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 200, 'subsample': 1.0}
Best RMSE: 18908.761248274193


In [6]:
# Evaluate the best model on the test set
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_test_pred = best_model.predict(X_test)

# Calculate RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {test_rmse}")


Test RMSE: 19437.809351154923


In [7]:
results.to_csv(folder_path + results_file, index=False)